In [ ]:
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
NUM_CLUSTERS = 16

# Load color counts

In [ ]:
counts = np.load('counts.npy')
identifiers = np.arange(256 * 256)
identifiers = identifiers.reshape(256, 256)
counts = counts[identifiers]

# for color ab (in LAB color space)
# counts[a, b] is the number it is seen

# Cluster colors

In [ ]:
kmeans = KMeans(n_clusters=NUM_CLUSTERS, n_init=1000)

y, x = np.where(counts > 0)
weights = counts[y, x]
colors = np.stack([y, x], axis=1)

kmeans.fit(colors, sample_weight=weights)

In [ ]:
centers = kmeans.cluster_centers_
q = np.quantile(counts, 0.9)

plt.matshow(np.clip(counts, 0, q))
plt.xlabel('a')
plt.ylabel('b')
plt.scatter(centers[:, 1], centers[:, 0]);

# Create a color to integer mapping

In [ ]:
x, y = np.meshgrid(np.arange(256), np.arange(256))
colors = np.stack([y, x], axis=2)  # shape [256, 256, 2]

In [ ]:
distance = ((np.expand_dims(colors, 2) - centers)**2).sum(3)  # shape [256, 256, NUM_CLUSTERS]
# distance[a, b, c] is a distance from color ab to cluster center c

In [ ]:
encoder = distance.argmin(2).astype('uint8')
# encoder[a, b] is an integer code for color ab

In [ ]:
decoder = []
for i in range(NUM_CLUSTERS):
    d = distance[:, :, i]
    decoder.append(np.unravel_index(d.argmin(), d.shape))

decoder = np.array(decoder, dtype='uint8')
# decoder[i] is a color for code i

In [ ]:
np.save('encoder.npy', encoder)
np.save('decoder.npy', decoder)

# Test color quantization

In [ ]:
import cv2
MIN_DIMENSION = 256


def quantize(path, result_path):
    
    cap = cv2.VideoCapture(path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    min_dimension = min(width, height)
    scaler = MIN_DIMENSION/min_dimension
    new_size = (int(width * scaler), int(height * scaler))
    
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(result_path, fourcc, fps, new_size)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, new_size, interpolation=cv2.INTER_CUBIC)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
            h, w, _ = frame.shape

            gray = frame[:, :, 0]
            frame = frame[:, :, 1:]
            frame = frame.reshape(-1, 2)
            a, b = frame[:, 0], frame[:, 1]

            gray = 255 * np.ones_like(gray)
            codes = encoder[a, b]
            frame = decoder[codes].reshape(h, w, 2)
            frame = np.concatenate([np.expand_dims(gray, 2), frame], axis=2)
            frame = cv2.cvtColor(frame, cv2.COLOR_LAB2RGB)            
            out.write(frame)
        else:
            break

    cap.release()
    out.release()

In [ ]:
quantize('data/videos/v_-xDx4qvX3KQ.mp4', 'result.avi')
# make the result video smaller by using:
# ffmpeg -i result.avi -c:v libx264 -c:a copy result.mp4 -y